#### Importing Libraries

In [1]:
import numpy as np
import gzip
import os
import pickle
import two_layer_net
import common
from PIL import Image

#### Sigmoid Function

In [2]:
def sigmoid(a):
    return 1/(1 + np.exp(-a))

#### Softmax Function

In [3]:
def softmax(a):
    c = np.max(a)
    a = np.exp(a - c)
    s = np.sum(a)
    return a/s 



In [4]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0] 
    return -np.sum(t * np.log(y + 1e-7))/batch_size


In [5]:
def _numerical_gradient(f, x):
    h = 1e-4

    grad = np.zeros_like(x) 

    for idx in range(x.size):
        # save x[idx]
        tmp = x[idx]

        # for f(x + h)
        x[idx] = tmp + h
        fh1 = f(x)

        # for f(x - h)
        x[idx] = tmp - h
        fh2 = f(x)

        grad[idx] = (fh1 - fh2)/(2*h)
        # restore x[idx]
        x[idx] = tmp
    return grad


In [6]:
def numerical_gradient(f, x):
    if x.ndim == 1:
        return _numerical_gradient(f, x)
    else:
        grad = np.zeros_like(x)
        for idx, x in enumerate(x):
            grad[idx] = _numerical_gradient(f, x)
        return grad


In [7]:
def predict(self, x):
        w1, w2 = self.params['w1'], self.params['w2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        return y


In [8]:
def loss(self, x, t):
    y = self.predict(x)
    return cross_entropy_error(y, t)

#### Defining Mnist Clas

In [9]:

class MnistData():   
    image_size = 784  # 28x28
    image_dim = (1, 28, 28)
    train_num = 60000
    test_num  = 10000
    key_file = {
        'train_images': 'C:/Users/13132/Documents/Pattern_Recoginition_NN/Homework_2/ece5831-2023-assignment-2/ece5831-2023-assignment-2/Assignment_6/Train && Test_Files/train-images-idx3-ubyte.gz',
        'train_labels': 'C:/Users/13132/Documents/Pattern_Recoginition_NN/Homework_2/ece5831-2023-assignment-2/ece5831-2023-assignment-2/Assignment_6/Train && Test_Files/train-labels-idx1-ubyte.gz',
        'test_images':  'C:/Users/13132/Documents/Pattern_Recoginition_NN/Homework_2/ece5831-2023-assignment-2/ece5831-2023-assignment-2/Assignment_6/Train && Test_Files/t10k-images-idx3-ubyte.gz',
        'test_labels':  'C:/Users/13132/Documents/Pattern_Recoginition_NN/Homework_2/ece5831-2023-assignment-2/ece5831-2023-assignment-2/Assignment_6/Train && Test_Files/t10k-labels-idx1-ubyte.gz'
    }

    def __init__(self):
        pass
    
    def _load_images(self, file_name):
        with gzip.open(file_name, 'rb') as f:
            images = np.frombuffer(f.read(), np.uint8, offset=16)
        images = images.reshape(-1, self.image_size)

        print('Done with loading images: ', file_name)    
        return images

    def _load_labels(self, file_name):
        with gzip.open(file_name, 'rb') as f: 
            labels = np.frombuffer(f.read(), np.uint8, offset=8)
        
        print('Done with loading labels: ', file_name)    
        return labels
    
    def _change_one_hot_label(self, x):
        t = np.zeros((x.size, 10))
        for idx, row in enumerate(t):
            row[x[idx]] = 1

        return t

    def load(self, normalize=True, flatten=True, one_hot_label=True):
        dataset = {}
        dataset['train_images'] = self._load_images(self.key_file['train_images'])
        dataset['train_labels'] = self._load_labels(self.key_file['train_labels'])
        dataset['test_images']  = self._load_images(self.key_file['test_images'])
        dataset['test_labels']  = self._load_labels(self.key_file['test_labels'])
        
        if normalize:
            for key in ('train_images', 'test_images'):
                dataset[key] = dataset[key].astype(np.float32)
                dataset[key] /= 255.0
    
        if one_hot_label:
            dataset['train_labels'] = self._change_one_hot_label(dataset['train_labels'])
            dataset['test_labels'] = self._change_one_hot_label(dataset['test_labels'])
    
        if not flatten:
             for key in ('train_images', 'test_images'):
                dataset[key] = dataset[key].reshape(-1, 1, 28, 28)
    
        return (dataset['train_images'], dataset['train_labels']), \
                (dataset['test_images'], dataset['test_labels'])

#### Two Layer Neural Network

In [10]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['w1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['w2'] = weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        w1, w2 = self.params['w1'], self.params['w2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, w1) + b1
        z1 = common.sigmoid(a1)

        a2 = np.dot(z1, w2) + b2
        y = common.softmax(a2)
        return y
        
    def loss(self, x, t):
        y = self.predict(x)
        return common.cross_entropy_error(y, t)
    
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)
        grads = {}
        grads['w1'] = common.numerical_gradient(loss_w, self.params['w1'])
        grads['b1'] = common.numerical_gradient(loss_w, self.params['b1'])
        grads['w2'] = common.numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = common.numerical_gradient(loss_w, self.params['b2'])

        return grads


#### Training the Model

In [ ]:
mnist_data = MnistData()
(x_train, t_train), (x_test, t_test) = mnist_data.load()

iters_num = 30
train_size = x_train.shape[0]
batch_size = 64
learning_rate = 0.01
train_loss = []
input_size = 28*28
Sai_Shanmukh_Varma_mnist_nn_model = two_layer_net.TwoLayerNet(input_size=input_size, hidden_size=100, output_size=10)

for i in range(iters_num):
    print("Current iteration is {}".format(i))
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    grad = Sai_Shanmukh_Varma_mnist_nn_model.numerical_gradient(x_batch, t_batch)
    for key in ('w1', 'b1', 'w2', 'b2'):
        Sai_Shanmukh_Varma_mnist_nn_model.params[key] -= learning_rate*grad[key]
loss = Sai_Shanmukh_Varma_mnist_nn_model.loss(x_batch, t_batch)
train_loss.append(loss)

with open('Sai_Shanmukh_Varma_mnist_nn_model.pkl', 'wb') as model:
    pickle.dump(Sai_Shanmukh_Varma_mnist_nn_model,model)
print("Pickle file completed")

Done with loading images:  C:/Users/13132/Documents/Pattern_Recoginition_NN/Homework_2/ece5831-2023-assignment-2/ece5831-2023-assignment-2/Assignment_6/Train && Test_Files/train-images-idx3-ubyte.gz
Done with loading labels:  C:/Users/13132/Documents/Pattern_Recoginition_NN/Homework_2/ece5831-2023-assignment-2/ece5831-2023-assignment-2/Assignment_6/Train && Test_Files/train-labels-idx1-ubyte.gz
Done with loading images:  C:/Users/13132/Documents/Pattern_Recoginition_NN/Homework_2/ece5831-2023-assignment-2/ece5831-2023-assignment-2/Assignment_6/Train && Test_Files/t10k-images-idx3-ubyte.gz
Done with loading labels:  C:/Users/13132/Documents/Pattern_Recoginition_NN/Homework_2/ece5831-2023-assignment-2/ece5831-2023-assignment-2/Assignment_6/Train && Test_Files/t10k-labels-idx1-ubyte.gz
Current iteration is 0
Current iteration is 1
Current iteration is 2
Current iteration is 3
Current iteration is 4
Current iteration is 5
Current iteration is 6
Current iteration is 7
Current iteration is 8

#### Testing the Model

In [11]:
def shape_image(images):
    my_image = np.array(images)
    my_image = my_image.reshape(784,)
    return my_image/255

def size_image(images):
    my_image_resized = None
    try:
        with Image.open(images) as image:
            my_image_grayscaled = image.convert("L")
            my_image_resized = my_image_grayscaled.resize((28,28))
    except Exception as e:
        print("Not able to process the image: {}".format(e))
    return my_image_resized

Trained_model = None
with open("Sai_Shanmukh_Varma_mnist_nn_model.pkl", 'rb') as model:
    Trained_model = pickle.load(model)

Test_image_size = size_image("Test_Images/9_1.png")
Test_image_shape = shape_image(Test_image_size)

y = Trained_model.predict(Test_image_shape)
print("My trained Model {}".format(y))
y_hat = np.argmax(y)
certainity = np.max(y)*100

print("Model output is predicted as {} with a certainity of {}".format(y_hat,certainity))

My trained Model [0.09853439 0.09911507 0.10371195 0.09808646 0.10840422 0.09695608
 0.08917946 0.0982873  0.09931603 0.10840902]
Model output is predicted as 9 with a certainity of 10.840902378521827
